# Test Benchmark

## Buffer

buffer store function (multi done scenario)

In [ ]:
from agent import AgentSAC
from attrdict import AttrDict
import torch
import yaml
from envs import toy

with open(f'configs/debug.yaml', "r") as stream:
  try:
    cfg = AttrDict(yaml.safe_load(stream))
  except yaml.YAMLError as exc:
    print(exc)
agent = AgentSAC(cfg)

In [ ]:
traj = agent.explore_vec_env()

In [ ]:
agent.buffer.data_parser(agent.buffer.data[8:12], 'info.ag')

In [ ]:
trans = agent.buffer.sample_batch(10, her_rate=1, indices=torch.arange(11, device='cuda:0'))

In [ ]:
obs_dict = agent.EP.obs_parser(trans.state[:10])
info_dict = agent.EP.info_parser(trans.info[:10])
obs_dict.ag, obs_dict.g, info_dict.step, trans.rew

In [ ]:
info_dict = agent.EP.info_parser(trans.info[:20])
info_dict.step

HER buffer

# Wandb

In [ ]:
import wandb
wandb.init(project="debug")